In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

In [2]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('..\\data\\UCI-fault\\Faults.csv',header=None)
pd.set_option('display.max_columns', None)
data.columns = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas", "Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults" ]



In [4]:
target = pd.DataFrame(data, columns=["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])
features = pd.DataFrame(data, columns=["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"])

In [5]:

tmp=[]
target_columns = ["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]
for i in range(len(target)):
    for j in range(0,7):
        if target.loc[i][j] == 1:
            tmp.append(target_columns[j])
targets = pd.DataFrame(tmp,columns=["target"])


In [6]:
features['target'] = tmp

In [7]:
train = features

In [12]:
target = ["target"]
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train","valid","test"], p = [.8, .1, .1], size = (train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [22]:
nunique = train.nunique() # 데이터에 고유값들의 수를 출력해주는 함수
types = train.dtypes

categorical_columns = []
categorical_dims = {}

for col in train.columns:
    if types[col] == 'object' or nunique[col] < 1000 :
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)
    
'''아래코드는 categorical embedding을 위한 전처리 부분 ( categorical 변수 dimension, idxs )'''
unused_feat = ['Set']
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

X_Minimum 962
X_Maximum 994
Pixels_Areas 920
X_Perimeter 399
Y_Perimeter 317
Minimum_of_Luminosity 161
Maximum_of_Luminosity 100
Length_of_Conveyer 84
TypeOfSteel_A300 2
TypeOfSteel_A400 2
Steel_Plate_Thickness 24
Square_Index 770
Outside_X_Index 454
Edges_X_Index 818
Edges_Y_Index 648
Outside_Global_Index 3
LogOfAreas 914
Log_X_Index 183
Log_Y_Index 217
Orientation_Index 918
SigmoidOfAreas 388
target 7
Set 3


In [23]:
cat_dims

[962,
 994,
 920,
 399,
 317,
 161,
 100,
 84,
 2,
 2,
 24,
 770,
 454,
 818,
 648,
 3,
 914,
 183,
 217,
 918,
 388]

In [24]:
cat_idxs

[0, 1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26]

In [14]:
features = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"]
target = ["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

KeyError: "None of [Index(['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps',\n       'Other_Faults'],\n      dtype='object')] are in the [columns]"

In [10]:
'''아래코드는 categorical embedding을 위한 전처리 부분 ( categorical 변수 dimension, idxs )'''
unused_feat = ['Set']
features = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"]
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [15]:
categorical_dims

{'Minimum_of_Luminosity': 161,
 'Maximum_of_Luminosity': 100,
 'Length_of_Conveyer': 84,
 'TypeOfSteel_A300': 2,
 'TypeOfSteel_A400': 2,
 'Steel_Plate_Thickness': 24,
 'Outside_Global_Index': 3,
 'Log_X_Index': 183,
 'target': 7,
 'Set': 3}

In [16]:
categorical_columns

['Minimum_of_Luminosity',
 'Maximum_of_Luminosity',
 'Length_of_Conveyer',
 'TypeOfSteel_A300',
 'TypeOfSteel_A400',
 'Steel_Plate_Thickness',
 'Outside_Global_Index',
 'Log_X_Index',
 'target',
 'Set']

In [11]:
cat_idxs

[8, 9, 10, 11, 12, 13, 20, 22]

In [12]:
cat_dims

[161, 100, 84, 2, 2, 24, 3, 183]

In [13]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
)

Device used : cuda


In [19]:
target = ' <=50K'

In [20]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

KeyError: ' <=50K'

In [10]:
'''
data = train
target = pd.DataFrame(data, columns=["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])
features = pd.DataFrame(data, columns=["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"])

x, x_test, y, y_test = train_test_split(features,target, test_size=0.2, shuffle=True, random_state=1004)
x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)
'''

'\ndata = train\ntarget = pd.DataFrame(data, columns=["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])\nfeatures = pd.DataFrame(data, columns=["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",\n              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",\n              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",\n              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",\n              "Luminosity_Index", "SigmoidOfAreas"])\n\nx, x_test, y, y_test = train_test_split(features,target, test_size=0.2, shuffle=True, random_state=1004)\nx_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)\n'

In [17]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

KeyError: 41

In [20]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetMultiTaskClassifier(**tabnet_params
                      )



Device used : cuda


In [21]:
x, x_test, y, y_test = train_test_split(features,target, test_size=0.2, shuffle=True, random_state=1004)
x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)

TypeError: Singleton array array(41) cannot be considered a valid collection.

In [18]:
x_train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Set
1247,609,621,6791894,6791907,101,16,13,11579,91,44,18,0,1,0,0.8943,0.3526,0.9231,0.0088,0.7500,1.0000,2,2.0043,10,1.1139,0.0769,-0.1044,0.2739,1
596,39,209,1434944,1435012,6676,240,128,688430,28,42,15,0,1,0,0.0575,0.4225,0.4000,0.1254,0.7083,0.5313,0,3.8245,135,1.8325,-0.6000,-0.1944,1.0000,1
231,114,127,1127920,1127927,60,14,8,6685,92,41,14,1,0,4,0.1684,0.3407,0.5385,0.0096,0.9286,0.8750,0,1.7781,11,0.8451,-0.4615,-0.1296,0.1965,1
1202,227,256,1593586,1593597,126,51,36,15951,102,61,21,1,0,5,0.3319,0.6050,0.3793,0.0212,0.5686,0.3056,0,2.1004,27,1.0414,-0.6207,-0.0110,0.5279,1
1339,347,364,1124976,1125024,390,51,50,46226,100,44,23,0,1,0,0.5058,0.5221,0.3542,0.0124,0.3333,0.9600,2,2.5911,15,1.6812,0.6458,-0.0740,0.9685,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,48,177,377537,377585,3985,166,99,413005,32,43,18,0,1,0,0.0705,0.3564,0.3721,0.0947,0.7771,0.4848,0,3.6004,99,1.6812,-0.6279,-0.1903,1.0000,1
1784,1194,1219,172567,172592,204,68,38,25491,104,56,16,0,1,0,0.2047,0.6736,1.0000,0.0184,0.3677,0.6579,1,2.3096,23,1.3979,0.0000,-0.0238,0.8958,2
1280,1613,1622,1752541,1752560,116,17,19,9105,55,14,60,1,0,5,0.0434,0.3216,0.4737,0.0054,0.5294,1.0000,2,2.0645,7,1.2787,0.5263,-0.3868,0.2942,0
483,39,192,1831860,1832017,11809,621,400,1367596,27,56,34,0,1,0,0.0557,0.5084,0.9745,0.1093,0.2464,0.3925,2,4.0722,121,2.1959,0.0255,-0.0952,1.0000,1


In [21]:
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()
y_test = y_test.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [23]:
x_train

array([[ 6.090000e+02,  6.210000e+02,  6.791894e+06, ..., -1.044000e-01,
         2.739000e-01,  1.000000e+00],
       [ 3.900000e+01,  2.090000e+02,  1.434944e+06, ..., -1.944000e-01,
         1.000000e+00,  1.000000e+00],
       [ 1.140000e+02,  1.270000e+02,  1.127920e+06, ..., -1.296000e-01,
         1.965000e-01,  1.000000e+00],
       ...,
       [ 1.613000e+03,  1.622000e+03,  1.752541e+06, ..., -3.868000e-01,
         2.942000e-01,  0.000000e+00],
       [ 3.900000e+01,  1.920000e+02,  1.831860e+06, ..., -9.520000e-02,
         1.000000e+00,  1.000000e+00],
       [ 9.800000e+01,  1.640000e+02,  1.895190e+05, ...,  3.530000e-02,
         9.760000e-01,  1.000000e+00]])

In [24]:
max_epochs = 100

# This illustrates the warm_start=False behaviour
#save_history = []

clf.fit(
        X_train=x_train, y_train=y_train,
        eval_set=[(x_train, y_train), (x_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )
#save_history.append(clf.history["valid_auc"])
    
#assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

ValueError: Please provide a list or np.array of weights for regression, multitask or pretraining: 1 given.